In [ ]:
#Copyright (c) 2019 Primoz Ravbar UCSB Licensed under BSD 2-Clause [see LICENSE for details] Written by Primoz Ravbar

# get ethograms from probabilities (prob outputs of NN)

import numpy as np
import scipy
from scipy import ndimage
from scipy import misc
import pickle
import pandas as pd
import time
import matplotlib.pyplot as plt
import cv2
import os
import matplotlib.colors as mcolors
import natsort
from PIL import Image

from sklearn.utils import shuffle

from ABRS_modules import smooth_1d



pathToABRS_GH_folder = '\\ABRS' #folder where this code file is in

topInputFolder = '' #output folder produced by video_to_etho code


modelName = 'modelConv2ABRS_3C_train_with_descendingcombinedwithothers_avi_10' #model trained with dusted flies

model = keras.models.load_model(modelName)

model.summary()


outputFolderEtho = pathToABRS_GH_folder + '\\Etho';

storeFrameRec = 0


normalizeByMax = 1;
thresholdMovement=10;

subfolderList = sorted(os.listdir(topInputFolder))

startNewEtho = 1 # start new ethograms 0 means NO
firstFolder = 0

numbSubfolders = 56

lengthEtho = 15*60*30;numbFiles = 580 #minutes*seconds*frames; number of files in output folder


if startNewEtho == 1:

    ethoMat = np.zeros((1, lengthEtho))
    probMat = np.zeros((10,lengthEtho))
    ethoMovementMat = np.zeros((1, lengthEtho))


for fld in range(firstFolder, numbSubfolders, 1):

    inputSubfolderName = subfolderList[fld]
    print (inputSubfolderName)

    inputSubfolderPathName = topInputFolder + '\\' + inputSubfolderName
    
    print (inputSubfolderPathName)

    fileList = sorted(os.listdir(inputSubfolderPathName))

    skipFilesNumb =1;
    skipFrameNumb=1;


    yi = np.zeros((1,10))
    yiVect = np.zeros((1,1))
 
    predictionsProbDataRec = np.zeros((10,lengthEtho)) 

    etho = np.zeros((1,lengthEtho))
    ethoMovement = np.zeros((1,lengthEtho))


    indIm = 0

    for fl in range(0, numbFiles-0, skipFilesNumb): #

        inputFileName = fileList[fl];

        fileDirPathInputName = inputSubfolderPathName + '\\' + inputFileName;

        with open(fileDirPathInputName, "rb") as f:
            dictEtho = pickle.load(f)

        predictionsProbRec = dictEtho["predictionsProbRec"] 

        maxMovRec = dictEtho['maxMovementRec'];


        for i in range(0, predictionsProbRec.shape[0]-0, skipFrameNumb):  
            
            if indIm < lengthEtho:
            
                predictionsProbData = np.transpose(predictionsProbRec[i,:])

                predictionsProbDataRec[:,indIm] = predictionsProbData


                etho[0,indIm] = np.argmax(predictionsProbData)

                ethoMovement[0,indIm] = maxMovRec[i]



                if maxMovRec[i] < thresholdMovement:
                    etho[0,indIm]=7

            indIm = indIm + 1    
    
    probMat = np.vstack((probMat,predictionsProbDataRec))
    ethoMat = np.vstack((ethoMat,etho))
    ethoMovementMat = np.vstack((ethoMovementMat,ethoMovement))